# Noveum Platform API Client - Complete Demo

This notebook demonstrates all 20 methods available in the Noveum Platform API Client.

## API Methods Coverage:
- **Trace Methods (6)**: Connection status, ingest, query, get trace, get spans
- **Dataset Methods (14)**: Create, list, get, update, delete, versions, version diff, items management

## Prerequisites:
- Set `NOVEUM_API_KEY` in your environment or `.env` file


## Setup


In [1]:
# Cell 0: Import required libraries
import json
from pprint import pprint

from dotenv import load_dotenv

from novaeval.noveum_platform import NoveumClient

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")


✓ Libraries imported successfully


In [2]:
# Cell 1: Configuration Variables
# Customize these values as needed

PROJECT_NAME = "noveum-api-wrapper-demo"
ENVIRONMENT = "development"
DATASET_NAME = "Sample Traces Wrapper Demo 13"
DATASET_SLUG = "reddit-traces-wrapper-demo-13"
DATASET_VERSION = "1.0.0"
SCORER_ID = "demo-quality-scorer"
NUM_TRACES = 10

print("Configuration:")
print(f"  Project: {PROJECT_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Dataset Slug: {DATASET_SLUG}")
print(f"  Version: {DATASET_VERSION}")
print(f"  Scorer ID: {SCORER_ID}")
print(f"  Number of traces: {NUM_TRACES}")


Configuration:
  Project: noveum-api-wrapper-demo
  Environment: development
  Dataset: Sample Traces Wrapper Demo 13
  Dataset Slug: reddit-traces-wrapper-demo-13
  Version: 1.0.0
  Scorer ID: demo-quality-scorer
  Number of traces: 10


In [3]:
# Cell 2: Load Test Data
print("Loading test data from sample_traces.json...")

with open("../test_data/sample_traces.json") as f:
    all_traces = json.load(f)

# Extract first NUM_TRACES traces
traces_sample = all_traces[:NUM_TRACES]

print(f"\n✓ Loaded {len(traces_sample)} traces for demonstration")
print("\nSample trace structure (first trace):")
print(f"  Trace ID: {traces_sample[0].get('trace_id', 'N/A')}")
print(f"  Span ID: {traces_sample[0].get('span_id', 'N/A')}")
print(f"  Name: {traces_sample[0].get('name', 'N/A')}")
print(f"  Project: {traces_sample[0].get('project', 'N/A')}")
print(f"  Environment: {traces_sample[0].get('environment', 'N/A')}")
print(f"  Status: {traces_sample[0].get('status', 'N/A')}")


Loading test data from sample_traces.json...

✓ Loaded 10 traces for demonstration

Sample trace structure (first trace):
  Trace ID: e5eebd9b-3f9e-463f-b8f3-96e20a1f771a
  Span ID: N/A
  Name: Sample Trace 1
  Project: noveum-api-wrapper-demo
  Environment: development
  Status: ok


In [4]:
# Cell 3: Initialize Noveum Client
print("Initializing Noveum Platform API Client...")

client = NoveumClient(
    # api_key is read from NOVEUM_API_KEY environment variable
)

print("✓ Client initialized successfully")
print(f"  Base URL: {client.base_url}")


Initializing Noveum Platform API Client...
✓ Client initialized successfully
  Base URL: https://noveum.ai


## Trace Methods (6 methods)

Demonstrating all trace-related API methods.


In [5]:
# Cell 4: Method 1 - get_connection_status()
print("=" * 60)
print("METHOD 1: get_connection_status()")
print("=" * 60)
print("\nChecking API connection status...\n")

connection_status = client.get_connection_status()
pprint(connection_status)


METHOD 1: get_connection_status()

Checking API connection status...

{'connected': True,
 'connection_source': 'environment_variables',
 'has_telemetry_plugin': False,
 'processing_time_ms': 14,
 'success': True,
 'tables_ready': True,
 'timestamp': '2025-10-22T18:56:33.047Z'}


In [6]:
# Cell 6: Method 2 - ingest_trace()
print("=" * 60)
print("METHOD 2: ingest_trace()")
print("=" * 60)
print("\nIngesting a single trace...\n")

# Prepare the first trace for ingestion
single_trace = traces_sample[0].copy()

# Update project and environment to our demo values
single_trace["project"] = PROJECT_NAME
single_trace["environment"] = ENVIRONMENT
if "trace_attributes" in single_trace:
    single_trace["trace_attributes"]["noveum.project"] = PROJECT_NAME
    single_trace["trace_attributes"]["noveum.environment"] = ENVIRONMENT

ingest_result = client.ingest_trace(single_trace)
print(f"✓ Successfully ingested trace: {single_trace['trace_id']}")
pprint(ingest_result)


METHOD 2: ingest_trace()

Ingesting a single trace...

✓ Successfully ingested trace: e5eebd9b-3f9e-463f-b8f3-96e20a1f771a
{'job_id': '17174',
 'message': 'Trace queued for batch processing',
 'processing_time_ms': 2,
 'success': True,
 'timestamp': '2025-10-22T18:56:33.560Z',
 'trace_id': 'e5eebd9b-3f9e-463f-b8f3-96e20a1f771a'}


In [7]:
# Cell 7: Method 3 - ingest_traces()
print("=" * 60)
print("METHOD 3: ingest_traces()")
print("=" * 60)
print("\nIngesting multiple traces in batch...\n")

# Prepare remaining traces for batch ingestion
batch_traces = []
for trace in traces_sample[1:]:
    trace_copy = trace.copy()
    trace_copy["project"] = PROJECT_NAME
    trace_copy["environment"] = ENVIRONMENT
    if "trace_attributes" in trace_copy:
        trace_copy["trace_attributes"]["noveum.project"] = PROJECT_NAME
        trace_copy["trace_attributes"]["noveum.environment"] = ENVIRONMENT
    batch_traces.append(trace_copy)

batch_result = client.ingest_traces(batch_traces)
print(f"✓ Successfully ingested {len(batch_traces)} traces")
pprint(batch_result)


METHOD 3: ingest_traces()

Ingesting multiple traces in batch...

✓ Successfully ingested 9 traces
{'job_id': '17175',
 'message': 'Batch traces queued for processing',
 'processing_time_ms': 12,
 'queued_count': 9,
 'success': True,
 'timestamp': '2025-10-22T18:56:34.079Z'}


In [8]:
# Cell 8: Method 4 - query_traces()
print("=" * 60)
print("METHOD 4: query_traces()")
print("=" * 60)
print("\nQuerying traces with filters...\n")

# Query traces with various filters
query_result = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=20,
    sort="start_time:desc",
    include_spans=False
)

print("Query parameters:")
print(f"  Project: {PROJECT_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print("  Size: 20")
print("  Sort: start_time:desc")
print("\nQuery results:")
pprint(query_result)

# Store trace IDs for later use
if "traces" in query_result and len(query_result["traces"]) > 0:
    sample_trace_id = query_result["traces"][0].get("trace_id")
    print(f"\n✓ Found {len(query_result.get('traces', []))} traces")
    print(f"  Sample trace ID for next demos: {sample_trace_id}")
else:
    sample_trace_id = traces_sample[0]["trace_id"]
    print(f"\n! No traces in query result, using original trace ID: {sample_trace_id}")


METHOD 4: query_traces()

Querying traces with filters...

Query parameters:
  Project: noveum-api-wrapper-demo
  Environment: development
  Size: 20
  Sort: start_time:desc

Query results:
{'pagination': {'has_more': False, 'limit': 20, 'offset': 0, 'total': 10},
 'processing_time_ms': 93,
 'success': True,
 'timestamp': '2025-10-22T18:56:34.470Z',
 'traces': [{'attributes': {},
             'created_at': '2025-10-21 13:09:01.180000000',
             'duration_ms': 532,
             'end_time': '2025-10-21 13:09:01.712000000',
             'environment': 'development',
             'error_count': 0,
             'metadata': {'custom_attributes': {},
                          'request_id': None,
                          'session_id': None,
                          'tags': {},
                          'user_id': None},
             'name': 'Sample Trace 5',
             'project': 'noveum-api-wrapper-demo',
             'sdk': {'name': 'noveum-python', 'version': '0.5.3'},
          

In [9]:
# Cell 8a: Method 4 - query_traces() with sort order options
print("=" * 60)
print("METHOD 4a: query_traces() - Sort Order Options")
print("=" * 60)
print("\nDemonstrating different sort orders...\n")
print("Use case: Finding longest/shortest running traces\n")

# Sort by duration descending (longest first)
print("1. Sort by duration (longest first - duration_ms:desc):")
traces_longest = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=5,
    sort="duration_ms:desc",
    include_spans=False
)

if "traces" in traces_longest:
    print(f"   Found {len(traces_longest['traces'])} traces")
    for trace in traces_longest["traces"][:3]:
        print(f"   - {trace.get('name')}: {trace.get('duration_ms')}ms")

# Sort by duration ascending (shortest first)
print("\n2. Sort by duration (shortest first - duration_ms:asc):")
traces_shortest = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=5,
    sort="duration_ms:asc",
    include_spans=False
)

if "traces" in traces_shortest:
    print(f"   Found {len(traces_shortest['traces'])} traces")
    for trace in traces_shortest["traces"][:3]:
        print(f"   - {trace.get('name')}: {trace.get('duration_ms')}ms")

print("\n✓ Sort demonstration complete")
print("  Available sort options: start_time:asc/desc, end_time:asc/desc, duration_ms:asc/desc")


METHOD 4a: query_traces() - Sort Order Options

Demonstrating different sort orders...

Use case: Finding longest/shortest running traces

1. Sort by duration (longest first - duration_ms:desc):
   Found 5 traces
   - Sample Trace 6: 3654ms
   - Sample Trace 10: 3002ms
   - Sample Trace 7: 2950ms

2. Sort by duration (shortest first - duration_ms:asc):
   Found 5 traces
   - Sample Trace 5: 532ms
   - Sample Trace 1: 556ms
   - Sample Trace 3: 906ms

✓ Sort demonstration complete
  Available sort options: start_time:asc/desc, end_time:asc/desc, duration_ms:asc/desc


In [10]:
# Cell 8b: Method 4 - query_traces() with status filter
print("=" * 60)
print("METHOD 4b: query_traces() - Status Filtering")
print("=" * 60)
print("\nFiltering traces by status...\n")
print("Use case: Debugging failed traces or monitoring success rates\n")

# Filter by status="ok"
print("1. Filter by status='ok':")
traces_ok = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    status="ok",
    size=5,
    include_spans=False
)

if "traces" in traces_ok:
    print(f"   Found {len(traces_ok['traces'])} successful traces")
    print(f"   Total ok traces: {traces_ok.get('pagination', {}).get('total', 'N/A')}")
    for trace in traces_ok["traces"][:3]:
        print(f"   - {trace.get('name')}: status={trace.get('status')}")

# Filter by status="error"
print("\n2. Filter by status='error':")
traces_error = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    status="error",
    size=5,
    include_spans=False
)

if "traces" in traces_error:
    print(f"   Found {len(traces_error['traces'])} error traces")
    print(f"   Total error traces: {traces_error.get('pagination', {}).get('total', 'N/A')}")
    if len(traces_error["traces"]) > 0:
        for trace in traces_error["traces"][:3]:
            print(f"   - {trace.get('name')}: status={trace.get('status')}")
    else:
        print("   No error traces found")

print("\n✓ Status filtering complete")
print("  Note: Use status filter for monitoring and debugging")


METHOD 4b: query_traces() - Status Filtering

Filtering traces by status...

Use case: Debugging failed traces or monitoring success rates

1. Filter by status='ok':
   Found 5 successful traces
   Total ok traces: 10
   - Sample Trace 8: status=ok
   - Sample Trace 10: status=ok
   - Sample Trace 7: status=ok

2. Filter by status='error':
   Found 0 error traces
   Total error traces: 0
   No error traces found

✓ Status filtering complete
  Note: Use status filter for monitoring and debugging


In [11]:
# Cell 8c: Method 4 - query_traces() with pagination
print("=" * 60)
print("METHOD 4c: query_traces() - Pagination Example")
print("=" * 60)
print("\nDemonstrating pagination with from_ and size...\n")
print("Use case: Efficiently paginating through large trace sets\n")

# First page
print("Fetching first page (from_=0, size=3):")
traces_page1 = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    from_=0,
    size=3,
    sort="start_time:desc",
    include_spans=False
)

if "traces" in traces_page1:
    print(f"  Page 1: {len(traces_page1['traces'])} traces")
    print(f"  Total available: {traces_page1.get('pagination', {}).get('total', 'N/A')}")
    for i, trace in enumerate(traces_page1["traces"], 1):
        print(f"  {i}. {trace.get('name')}")

# Second page
print("\nFetching second page (from_=3, size=3):")
traces_page2 = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    from_=3,
    size=3,
    sort="start_time:desc",
    include_spans=False
)

if "traces" in traces_page2:
    print(f"  Page 2: {len(traces_page2['traces'])} traces")
    for i, trace in enumerate(traces_page2["traces"], 1):
        print(f"  {i}. {trace.get('name')}")

print("\n✓ Pagination demo complete")
print("  Note: from_ is 0-based offset, calculate as page_number * size")


METHOD 4c: query_traces() - Pagination Example

Demonstrating pagination with from_ and size...

Use case: Efficiently paginating through large trace sets

Fetching first page (from_=0, size=3):
  Page 1: 3 traces
  Total available: 10
  1. Sample Trace 3
  2. Sample Trace 4
  3. Sample Trace 1

Fetching second page (from_=3, size=3):
  Page 2: 3 traces
  1. Sample Trace 9
  2. Sample Trace 3
  3. Sample Trace 4

✓ Pagination demo complete
  Note: from_ is 0-based offset, calculate as page_number * size


In [12]:
# Cell 8d: Method 4 - query_traces() with include_spans comparison
print("=" * 60)
print("METHOD 4e: query_traces() - Include Spans Comparison")
print("=" * 60)
print("\nComparing results with and without span data...\n")
print("Use case: Get detailed span information when debugging or analyzing trace details\n")

# Without spans
print("1. WITHOUT spans (include_spans=False):")
traces_no_spans = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=1,
    include_spans=False
)

if "traces" in traces_no_spans and len(traces_no_spans["traces"]) > 0:
    trace = traces_no_spans["traces"][0]
    print(f"   Trace: {trace.get('name')}")
    print(f"   Span count: {trace.get('span_count')}")
    print(f"   Spans included in response: {len(trace.get('spans', []))} spans")
    print(f"   Response size indicator: ~{len(str(trace))} characters")

# With spans
print("\n2. WITH spans (include_spans=True):")
traces_with_spans = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    size=1,
    include_spans=True
)

if "traces" in traces_with_spans and len(traces_with_spans["traces"]) > 0:
    trace = traces_with_spans["traces"][0]
    print(f"   Trace: {trace.get('name')}")
    print(f"   Span count: {trace.get('span_count')}")
    print(f"   Spans included in response: {len(trace.get('spans', []))} spans")
    print(f"   Response size indicator: ~{len(str(trace))} characters")
    if len(trace.get("spans", [])) > 0:
        print(f"   First span name: {trace['spans'][0].get('name')}")

print("\n✓ Include spans comparison complete")
print("  Note: include_spans=True significantly increases response size but provides detailed span data")


METHOD 4e: query_traces() - Include Spans Comparison

Comparing results with and without span data...

Use case: Get detailed span information when debugging or analyzing trace details

1. WITHOUT spans (include_spans=False):
   Trace: Sample Trace 5
   Span count: 5
   Spans included in response: 0 spans
   Response size indicator: ~618 characters

2. WITH spans (include_spans=True):
   Trace: Sample Trace 10
   Span count: 5
   Spans included in response: 0 spans
   Response size indicator: ~620 characters

✓ Include spans comparison complete
  Note: include_spans=True significantly increases response size but provides detailed span data


In [13]:
# Cell 8e: Method 4 - query_traces() with search_term
print("=" * 60)
print("METHOD 4f: query_traces() - Search Term Usage")
print("=" * 60)
print("\nDemonstrating text search across traces...\n")
print("Use case: Finding specific traces by name or content\n")

# Search for "Trace 1"
print("1. Search for 'Trace 1':")
traces_search1 = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    search_term="Trace 1",
    size=10,
    include_spans=False
)

if "traces" in traces_search1:
    print(f"   Found {len(traces_search1['traces'])} traces matching 'Trace 1'")
    print(f"   Total matches: {traces_search1.get('pagination', {}).get('total', 'N/A')}")
    for trace in traces_search1["traces"][:3]:
        print(f"   - {trace.get('name')}")

# Search for "Trace 3"
print("\n2. Search for 'Trace 3':")
traces_search2 = client.query_traces(
    project=PROJECT_NAME,
    environment=ENVIRONMENT,
    search_term="Trace 3",
    size=10,
    include_spans=False
)

if "traces" in traces_search2:
    print(f"   Found {len(traces_search2['traces'])} traces matching 'Trace 3'")
    print(f"   Total matches: {traces_search2.get('pagination', {}).get('total', 'N/A')}")
    for trace in traces_search2["traces"][:3]:
        print(f"   - {trace.get('name')}")

print("\n✓ Search term demonstration complete")
print("  Note: search_term performs text search across trace names and content")


METHOD 4f: query_traces() - Search Term Usage

Demonstrating text search across traces...

Use case: Finding specific traces by name or content

1. Search for 'Trace 1':
   Found 10 traces matching 'Trace 1'
   Total matches: 10
   - Sample Trace 8
   - Sample Trace 10
   - Sample Trace 7

2. Search for 'Trace 3':
   Found 10 traces matching 'Trace 3'
   Total matches: 10
   - Sample Trace 3
   - Sample Trace 4
   - Sample Trace 1

✓ Search term demonstration complete
  Note: search_term performs text search across trace names and content


In [14]:
# Cell 9: Method 5 - get_trace()
print("=" * 60)
print("METHOD 5: get_trace()")
print("=" * 60)
print(f"\nFetching specific trace by ID: {sample_trace_id}\n")

trace_detail = client.get_trace(sample_trace_id)
pprint(trace_detail)


METHOD 5: get_trace()

Fetching specific trace by ID: 77828ea1-8a55-4dee-9df3-6ce914ebeaab

{'data': {'attributes': {},
          'created_at': '2025-10-21 13:09:01.180000000',
          'duration_ms': 532,
          'end_time': '2025-10-21 13:09:01.712000000',
          'environment': 'development',
          'error_count': 0,
          'metadata': {'custom_attributes': {},
                       'request_id': None,
                       'session_id': None,
                       'tags': {},
                       'user_id': None},
          'name': 'Sample Trace 5',
          'project': 'noveum-api-wrapper-demo',
          'sdk': {'name': 'noveum-python', 'version': '0.5.3'},
          'span_count': 5,
          'spans': [{'attributes': {'operation.type': 'query',
                                    'request.size': 5136,
                                    'service.name': 'service-1',
                                    'user.id': 'user-4857'},
                     'duration_ms': 14

In [15]:
# Cell 10: Method 6 - get_trace_spans()
print("=" * 60)
print("METHOD 6: get_trace_spans()")
print("=" * 60)
print(f"\nFetching spans for trace: {sample_trace_id}\n")

trace_spans = client.get_trace_spans(sample_trace_id)
pprint(trace_spans)

if "spans" in trace_spans:
    print(f"\n✓ Found {len(trace_spans['spans'])} spans")


METHOD 6: get_trace_spans()

Fetching spans for trace: 77828ea1-8a55-4dee-9df3-6ce914ebeaab

{'spans': [{'attributes': {'operation.type': 'query',
                           'request.size': 5136,
                           'service.name': 'service-1',
                           'user.id': 'user-4857'},
            'duration_ms': 142,
            'end_time': '2025-10-21 13:09:01.322000000',
            'events': [],
            'links': [],
            'name': 'Sample Trace 5_span_1',
            'parent_span_id': '',
            'span_id': '4b4a6371-353b-4412-bc1a-7aedcb8c5423',
            'start_time': '2025-10-21 13:09:01.180000000',
            'status': 'ok',
            'status_message': '',
            'trace_id': '77828ea1-8a55-4dee-9df3-6ce914ebeaab'},
           {'attributes': {'operation.type': 'subscription',
                           'request.size': 8956,
                           'service.name': 'service-2',
                           'user.id': 'user-4734'},
          

In [16]:
# Cell 12a: Method 8 - list_datasets() with visibility filter
print("=" * 60)
print("METHOD 8a: list_datasets() - Filter by Visibility (org)")
print("=" * 60)
print("\nListing only organization-scoped datasets...\n")
print("Use case: View datasets shared within your organization\n")

datasets_org = client.list_datasets(
    limit=5,
    offset=0,
    visibility="org",
    includeVersions=False
)

pprint(datasets_org)

if "datasets" in datasets_org:
    print(f"\n✓ Found {len(datasets_org['datasets'])} org-scoped datasets (showing first 5)")
    print(f"  Total org datasets: {datasets_org.get('total', 'N/A')}")


METHOD 8a: list_datasets() - Filter by Visibility (org)

Listing only organization-scoped datasets...

Use case: View datasets shared within your organization

{'datasets': [{'avg_quality_score': 0,
               'created_at': '2025-09-30 11:55:51.486000000',
               'created_by': 'system',
               'current_release': '0.0.0',
               'custom_attributes': '{}',
               'dataset_type': 'agent',
               'description': '',
               'environment': 'development',
               'id': 'be0eaeca-4532-422a-ba8b-69915c373fe5',
               'is_active': 1,
               'items_by_type': '{}',
               'name': 'Agent Dataset',
               'next_release': '1.0.0',
               'organization_id': 'magic-api',
               'organization_slug': 'magic-api',
               'project_id': 'default',
               'slug': 'agent',
               'tags': '[]',
               'total_items': '0',
               'total_size_bytes': '0',
              

In [17]:
# Cell 12b: Method 8 - list_datasets() with private visibility filter
print("=" * 60)
print("METHOD 8b: list_datasets() - Filter by Visibility (private)")
print("=" * 60)
print("\nListing only private datasets...\n")
print("Use case: View datasets that are private to you only\n")

datasets_private = client.list_datasets(
    limit=5,
    offset=0,
    visibility="private",
    includeVersions=False
)

pprint(datasets_private)

if "datasets" in datasets_private:
    print(f"\n✓ Found {len(datasets_private['datasets'])} private datasets (showing first 5)")
    print(f"  Total private datasets: {datasets_private.get('total', 'N/A')}")
else:
    print("\n! No private datasets found")


METHOD 8b: list_datasets() - Filter by Visibility (private)

Listing only private datasets...

Use case: View datasets that are private to you only

{'datasets': [], 'limit': 5, 'offset': 0, 'success': True, 'total': 0}

✓ Found 0 private datasets (showing first 5)
  Total private datasets: 0


In [18]:
# Cell 12c: Method 8 - list_datasets() with pagination
print("=" * 60)
print("METHOD 8c: list_datasets() - Pagination Example")
print("=" * 60)
print("\nDemonstrating pagination with limit and offset...\n")
print("Use case: Efficiently navigate through large dataset lists\n")

# First page
print("Fetching first page (limit=5, offset=0):")
datasets_page1 = client.list_datasets(
    limit=5,
    offset=0,
    visibility="org",
    includeVersions=False
)

if "datasets" in datasets_page1:
    print(f"  Page 1: {len(datasets_page1['datasets'])} datasets")
    print(f"  Total available: {datasets_page1.get('total', 'N/A')}")
    if len(datasets_page1["datasets"]) > 0:
        print(f"  First dataset: {datasets_page1['datasets'][0].get('name', 'N/A')}")

# Second page
print("\nFetching second page (limit=5, offset=5):")
datasets_page2 = client.list_datasets(
    limit=5,
    offset=5,
    visibility="org",
    includeVersions=False
)

if "datasets" in datasets_page2:
    print(f"  Page 2: {len(datasets_page2['datasets'])} datasets")
    if len(datasets_page2["datasets"]) > 0:
        print(f"  First dataset on page 2: {datasets_page2['datasets'][0].get('name', 'N/A')}")

print("\n✓ Pagination demo complete")
print("  Note: Use offset = page_number * limit for specific pages")


METHOD 8c: list_datasets() - Pagination Example

Demonstrating pagination with limit and offset...

Use case: Efficiently navigate through large dataset lists

Fetching first page (limit=5, offset=0):
  Page 1: 5 datasets
  Total available: 38
  First dataset: Agent Dataset

Fetching second page (limit=5, offset=5):
  Page 2: 5 datasets
  First dataset on page 2: Sample Traces Wrapper Demo 12

✓ Pagination demo complete
  Note: Use offset = page_number * limit for specific pages


In [19]:
# Cell 12d: Method 8 - list_datasets() with includeVersions comparison
print("=" * 60)
print("METHOD 8d: list_datasets() - Include Versions Comparison")
print("=" * 60)
print("\nComparing results with and without version information...\n")
print("Use case: Get version history when you need detailed dataset evolution info\n")

# Without versions
print("1. WITHOUT versions (includeVersions=False):")
datasets_no_versions = client.list_datasets(
    limit=2,
    offset=0,
    visibility="org",
    includeVersions=False
)

if "datasets" in datasets_no_versions and len(datasets_no_versions["datasets"]) > 0:
    first_dataset = datasets_no_versions["datasets"][0]
    print(f"   Dataset: {first_dataset.get('name', 'N/A')}")
    print(f"   Has 'versions' field: {'versions' in first_dataset}")
    if "versions" in first_dataset:
        print(f"   Versions: {first_dataset['versions']}")

# With versions
print("\n2. WITH versions (includeVersions=True):")
datasets_with_versions = client.list_datasets(
    limit=2,
    offset=0,
    visibility="org",
    includeVersions=True
)

if "datasets" in datasets_with_versions and len(datasets_with_versions["datasets"]) > 0:
    first_dataset = datasets_with_versions["datasets"][0]
    print(f"   Dataset: {first_dataset.get('name', 'N/A')}")
    print(f"   Has 'versions' field: {'versions' in first_dataset}")
    if "versions" in first_dataset:
        print(f"   Versions: {first_dataset['versions']}")
        print(f"   Number of versions: {len(first_dataset['versions'])}")

print("\n✓ Comparison complete")
print("  Note: includeVersions=True adds version history to each dataset")


METHOD 8d: list_datasets() - Include Versions Comparison

Comparing results with and without version information...

Use case: Get version history when you need detailed dataset evolution info

1. WITHOUT versions (includeVersions=False):
   Dataset: Agent Dataset
   Has 'versions' field: True
   Versions: ['1.0.0', '0.0.0']

2. WITH versions (includeVersions=True):
   Dataset: Agent Dataset
   Has 'versions' field: True
   Versions: ['1.0.0', '0.0.0']
   Number of versions: 2

✓ Comparison complete
  Note: includeVersions=True adds version history to each dataset


## Dataset Methods (14 methods)

Demonstrating all dataset-related API methods.


In [20]:
# Cell 11: Method 7 - create_dataset()
print("=" * 60)
print("METHOD 7: create_dataset()")
print("=" * 60)
print("\nCreating a new dataset...\n")

dataset = client.create_dataset(
    name=DATASET_NAME,
    slug=DATASET_SLUG,
    description="Demo dataset containing Sample traces for API demonstration",
    visibility="org",
    dataset_type="agent",
    environment=ENVIRONMENT,
    tags=["demo", "Sample", "traces"],
    custom_attributes={
        "source": "sample_traces.json",
        "purpose": "API demonstration"
    }
)

print("✓ Dataset created successfully")
pprint(dataset)


METHOD 7: create_dataset()

Creating a new dataset...

✓ Dataset created successfully
{'dataset': {'avg_quality_score': 0,
             'created_at': '2025-10-22 18:56:41.538000000',
             'created_by': 'system',
             'current_release': '0.0.0',
             'custom_attributes': '{"source":"sample_traces.json","purpose":"API '
                                  'demonstration"}',
             'dataset_type': 'agent',
             'description': 'Demo dataset containing Sample traces for API '
                            'demonstration',
             'environment': 'development',
             'id': '00f5ec5e-8401-4cbd-85c8-9b7a613eb8c0',
             'is_active': 1,
             'items_by_type': '{}',
             'name': 'Sample Traces Wrapper Demo 13',
             'next_release': '0.0.1',
             'organization_id': 'magic-api',
             'organization_slug': 'magic-api',
             'project_id': 'default',
             'slug': 'reddit-traces-wrapper-demo-13',


In [21]:
# Cell 12: Method 8 - list_datasets()
print("=" * 60)
print("METHOD 8: list_datasets()")
print("=" * 60)
print("\nListing all datasets...\n")

datasets_list = client.list_datasets(
    limit=20,
    offset=0,
    visibility="org",
    includeVersions=True
)

pprint(datasets_list)

if "datasets" in datasets_list:
    print(f"\n✓ Found {len(datasets_list['datasets'])} datasets")


METHOD 8: list_datasets()

Listing all datasets...

{'datasets': [{'avg_quality_score': 0,
               'created_at': '2025-09-30 11:55:51.486000000',
               'created_by': 'system',
               'current_release': '0.0.0',
               'custom_attributes': '{}',
               'dataset_type': 'agent',
               'description': '',
               'environment': 'development',
               'id': 'be0eaeca-4532-422a-ba8b-69915c373fe5',
               'is_active': 1,
               'items_by_type': '{}',
               'name': 'Agent Dataset',
               'next_release': '1.0.0',
               'organization_id': 'magic-api',
               'organization_slug': 'magic-api',
               'project_id': 'default',
               'slug': 'agent',
               'tags': '[]',
               'total_items': '0',
               'total_size_bytes': '0',
               'updated_at': '2025-10-21 05:21:15.020000000',
               'validation_distribution': '{}',
            

In [22]:
# Cell 13: Method 9 - get_dataset()
print("=" * 60)
print("METHOD 9: get_dataset()")
print("=" * 60)
print(f"\nFetching dataset by slug: {DATASET_SLUG}\n")

dataset_detail = client.get_dataset(DATASET_SLUG)
pprint(dataset_detail)


METHOD 9: get_dataset()

Fetching dataset by slug: reddit-traces-wrapper-demo-13

{'dataset': {'avg_quality_score': 0,
             'created_at': '2025-10-22 18:56:41.538000000',
             'created_by': 'system',
             'current_release': '0.0.0',
             'custom_attributes': '{"source":"sample_traces.json","purpose":"API '
                                  'demonstration"}',
             'dataset_type': 'agent',
             'description': 'Demo dataset containing Sample traces for API '
                            'demonstration',
             'environment': 'development',
             'id': '00f5ec5e-8401-4cbd-85c8-9b7a613eb8c0',
             'is_active': 1,
             'items_by_type': '{}',
             'name': 'Sample Traces Wrapper Demo 13',
             'next_release': '0.0.1',
             'organization_id': 'magic-api',
             'organization_slug': 'magic-api',
             'project_id': 'default',
             'slug': 'reddit-traces-wrapper-demo-13',
    

In [23]:
# Cell 14: Method 10 - update_dataset()
print("=" * 60)
print("METHOD 10: update_dataset()")
print("=" * 60)
print(f"\nUpdating dataset: {DATASET_SLUG}\n")

updated_dataset = client.update_dataset(
    slug=DATASET_SLUG,
    description="Updated: Demo dataset with Sample traces - now with enhanced metadata",
    tags=["demo", "Sample", "traces", "updated"],
    custom_attributes={
        "source": "sample_traces.json",
        "purpose": "API demonstration",
        "updated": True,
        "version_info": "Enhanced with additional metadata"
    }
)

print("✓ Dataset updated successfully")
pprint(updated_dataset)


METHOD 10: update_dataset()

Updating dataset: reddit-traces-wrapper-demo-13

✓ Dataset updated successfully
{'dataset': {'avg_quality_score': 0,
             'created_at': '2025-10-22 18:56:41.538000000',
             'created_by': 'system',
             'current_release': '0.0.0',
             'custom_attributes': '{"source":"sample_traces.json","purpose":"API '
                                  'demonstration"}',
             'dataset_type': 'agent',
             'description': 'Updated: Demo dataset with Sample traces - now '
                            'with enhanced metadata',
             'environment': 'development',
             'id': '00f5ec5e-8401-4cbd-85c8-9b7a613eb8c0',
             'is_active': 1,
             'items_by_type': '{}',
             'name': 'Sample Traces Wrapper Demo 13',
             'next_release': '0.0.1',
             'organization_id': 'magic-api',
             'organization_slug': 'magic-api',
             'project_id': 'default',
             'slug': 

In [24]:
# Cell 15: Method 11 - create_dataset_version()
print("=" * 60)
print("METHOD 11: create_dataset_version()")
print("=" * 60)
print(f"\nCreating version {DATASET_VERSION} for dataset: {DATASET_SLUG}\n")

version_data = {
    "version": DATASET_VERSION,
    "description": "Initial version with 10 Sample trace samples",
    "tags": ["v1", "initial"],
    "custom_attributes": {
        "trace_count": NUM_TRACES,
        "created_by": "demo_script"
    }
}

version = client.create_dataset_version(DATASET_SLUG, version_data)
print(f"✓ Version {DATASET_VERSION} created successfully")
pprint(version)


METHOD 11: create_dataset_version()

Creating version 1.0.0 for dataset: reddit-traces-wrapper-demo-13

✓ Version 1.0.0 created successfully
{'success': True, 'version': '1.0.0'}


In [25]:
# Cell 16: Method 12 - list_dataset_versions()
print("=" * 60)
print("METHOD 12: list_dataset_versions()")
print("=" * 60)
print(f"\nListing all versions for dataset: {DATASET_SLUG}\n")

versions_list = client.list_dataset_versions(DATASET_SLUG)
pprint(versions_list)

if "versions" in versions_list:
    print(f"\n✓ Found {len(versions_list['versions'])} versions")


METHOD 12: list_dataset_versions()

Listing all versions for dataset: reddit-traces-wrapper-demo-13

{'current_release': '0.0.0',
 'next_release': '0.0.1',
 'success': True,
 'versions': ['0.0.1', '0.0.0']}

✓ Found 2 versions


In [26]:
# Cell 17: Method 13 - get_dataset_version()
print("=" * 60)
print("METHOD 13: get_dataset_version()")
print("=" * 60)
print(f"\nFetching version {DATASET_VERSION} for dataset: {DATASET_SLUG}\n")

version_detail = client.get_dataset_version(DATASET_SLUG, DATASET_VERSION)
pprint(version_detail)


METHOD 13: get_dataset_version()

Fetching version 1.0.0 for dataset: reddit-traces-wrapper-demo-13

{'current_release': '0.0.0',
 'exists': False,
 'is_current_release': False,
 'is_next_release': False,
 'item_count': 0,
 'next_release': '0.0.1',
 'status': 'nonexistent',
 'success': True,
 'version': '1.0.0'}


In [27]:
# Cell 17a: Method 14 - get_dataset_versions_diff()
print("=" * 60)
print("METHOD 14: get_dataset_versions_diff()")
print("=" * 60)
print(f"\nGetting diff between current_release and next_release for: {DATASET_SLUG}\n")

versions_diff = client.get_dataset_versions_diff(DATASET_SLUG)
pprint(versions_diff)

print("\n✓ Version diff retrieved successfully")
print("This shows changes between current_release and next_release versions")


METHOD 14: get_dataset_versions_diff()

Getting diff between current_release and next_release for: reddit-traces-wrapper-demo-13

{'changes': {'added': [], 'deleted': [], 'modified': []},
 'current_release': '0.0.0',
 'current_release_count': 0,
 'next_release': '0.0.1',
 'next_release_count': 0,
 'success': True,
 'summary': {'added': 0, 'deleted': 0, 'modified': 0, 'total_changes': 0}}

✓ Version diff retrieved successfully
This shows changes between current_release and next_release versions


In [28]:
# Cell 18: Transform traces into dataset items
print("=" * 60)
print("PREPARING DATASET ITEMS")
print("=" * 60)
print("\nTransforming traces into dataset item format...\n")

dataset_items = []
for i, trace in enumerate(traces_sample):
    item = {
        "item_key": f"trace_{i+1:03d}",
        "item_type": "agent_trace",
        "content": {
            **trace,
            "trace_id": trace.get("trace_id"),
            "span_id": trace.get("span_id"),
            "name": trace.get("name"),
            "start_time": trace.get("start_time"),
            "end_time": trace.get("end_time"),
            "duration_ms": trace.get("duration_ms"),
            "status": trace.get("status"),
            "attributes": trace.get("attributes", {})
        },
        "metadata": {
            "project": trace.get("project", PROJECT_NAME),
            "environment": trace.get("environment", ENVIRONMENT),
            "item_index": i + 1
        }
    }
    dataset_items.append(item)

print(f"✓ Prepared {len(dataset_items)} dataset items")
print("\nSample item structure:")
pprint(dataset_items[0])


PREPARING DATASET ITEMS

Transforming traces into dataset item format...

✓ Prepared 10 dataset items

Sample item structure:
{'content': {'attributes': {},
             'duration_ms': 556,
             'end_time': '2025-10-21T13:09:01.736210Z',
             'environment': 'development',
             'name': 'Sample Trace 1',
             'project': 'noveum-api-wrapper-demo',
             'sdk': {'name': 'noveum-python', 'version': '0.5.3'},
             'span_count': 2,
             'span_id': None,
             'spans': [{'attributes': {'operation.type': 'query',
                                       'request.size': 5755,
                                       'service.name': 'service-1',
                                       'user.id': 'user-9725'},
                        'duration_ms': 293,
                        'end_time': '2025-10-21T13:09:01.473210Z',
                        'name': 'Sample Trace 1_span_1',
                        'span_id': 'c889ca64-04c8-4c52-b2d0-f159c06

In [29]:
# Cell 19: Method 15 - add_dataset_items()
print("=" * 60)
print("METHOD 15: add_dataset_items()")
print("=" * 60)
print(f"\nAdding {len(dataset_items)} items to dataset: {DATASET_SLUG}\n")
print("Note: Items are automatically added to the 'next_release' version\n")

add_items_result = client.add_dataset_items(
    dataset_slug=DATASET_SLUG,
    items=dataset_items
)

print(f"✓ Successfully added {len(dataset_items)} items")
pprint(add_items_result)


METHOD 15: add_dataset_items()

Adding 10 items to dataset: reddit-traces-wrapper-demo-13

Note: Items are automatically added to the 'next_release' version

✓ Successfully added 10 items
{'created': 10, 'success': True}


In [30]:
# Cell 20: Method 16 - publish_dataset_version()
print("=" * 60)
print("METHOD 16: publish_dataset_version()")
print("=" * 60)

print(f"\nPublishing next_release version for dataset: {DATASET_SLUG}\n")
print("Note: Automatically publishes 'next_release' and increments version number\n")

publish_result = client.publish_dataset_version(DATASET_SLUG)
print("✓ Version published successfully")
pprint(publish_result)


METHOD 16: publish_dataset_version()

Publishing next_release version for dataset: reddit-traces-wrapper-demo-13

Note: Automatically publishes 'next_release' and increments version number

✓ Version published successfully
{'current_release': '0.0.1', 'next_release': '0.0.2', 'success': True}


In [31]:
# Cell 21: Method 17 - list_dataset_items()
print("=" * 60)
print("METHOD 17: list_dataset_items()")
print("=" * 60)
print(f"\nListing items from dataset: {DATASET_SLUG}\n")
print("Note: New optional parameters available: item_type, search, sort_by, sort_order\n")

items_list = client.list_dataset_items(
    dataset_slug=DATASET_SLUG,
    limit=20,
    offset=0
)

pprint(items_list)

if "items" in items_list:
    print(f"\n✓ Found {len(items_list['items'])} items")
    # Store first item key for later use
    if len(items_list["items"]) > 0:
        sample_item_key = items_list["items"][0].get("item_key", "trace_001")
        sample_item_id = items_list["items"][0].get("id", items_list["items"][0].get("item_key", "trace_001"))
        print(f"  Sample item key: {sample_item_key}")
        print(f"  Sample item ID: {sample_item_id}")
else:
    sample_item_key = "trace_001"
    sample_item_id = "trace_001"


METHOD 17: list_dataset_items()

Listing items from dataset: reddit-traces-wrapper-demo-13

Note: New optional parameters available: item_type, search, sort_by, sort_order

{'items': [{'agent_exit': '',
            'agent_name': '',
            'agent_response': '',
            'agent_role': '',
            'agent_task': '',
            'content': '',
            'conversation_context': '{}',
            'conversation_id': '',
            'created_at': '2025-10-22 18:56:44.114000000',
            'criteria': '',
            'custom_attributes': '{}',
            'dataset_id': '00f5ec5e-8401-4cbd-85c8-9b7a613eb8c0',
            'dataset_slug': 'reddit-traces-wrapper-demo-13',
            'deleted_at_date': None,
            'deleted_at_version': None,
            'evaluation_context': '{}',
            'exit_status': '',
            'expected_output': '',
            'expected_tool_call': '',
            'ground_truth': '',
            'input_text': '',
            'item_hash': 'f940adf

In [32]:
# # Cell 22: Method 19 - get_dataset_item() currently doesnt work
# print("=" * 60)
# print("METHOD 19: get_dataset_item()")
# print("=" * 60)
# print(f"\nFetching specific item: {sample_item_key}\n")

# item_detail = client.get_dataset_item(DATASET_SLUG, sample_item_key)
# pprint(item_detail)


In [33]:
# Cell 23: Method 18 - delete_dataset_item()
print("=" * 60)
print("METHOD 18: delete_dataset_item()")
print("=" * 60)
print(f"\nDeleting specific item: {sample_item_id}\n")

delete_item_result = client.delete_dataset_item(DATASET_SLUG, sample_item_id)
print(f"✓ Item {sample_item_id} deleted successfully")
pprint(delete_item_result)


METHOD 18: delete_dataset_item()

Deleting specific item: trace_001

✓ Item trace_001 deleted successfully
{'success': True}


In [34]:
# Cell 24: Method 19 - delete_all_dataset_items() - Demo only
print("=" * 60)
print("METHOD 19: delete_all_dataset_items()")
print("=" * 60)
print("Demonstrating delete_all_dataset_items()...")

# Note: This method requires specific item IDs to delete
# For demo purposes, we will use some example item IDs
example_item_ids = ["item-1", "item-2", "item-3"]
print(f"Attempting to delete items: {example_item_ids}")

try:
    delete_result = client.delete_all_dataset_items(DATASET_SLUG, example_item_ids)
    print(f"✓ Deletion result: {delete_result}")
except Exception as e:
    print(f"⚠ Deletion failed (expected if items do not exist): {type(e).__name__}: {e}")

print("\nNote: delete_all_dataset_items() requires item_ids as second parameter (required)")
print("The method will throw ValidationError if no item_ids are provided")


METHOD 19: delete_all_dataset_items()
Demonstrating delete_all_dataset_items()...
Attempting to delete items: ['item-1', 'item-2', 'item-3']
✓ Deletion result: {'success': True, 'deleted': 3}

Note: delete_all_dataset_items() requires item_ids as second parameter (required)
The method will throw ValidationError if no item_ids are provided


In [35]:
# Cell 32: Method 20 - delete_dataset()
print("=" * 60)
print("METHOD 20: delete_dataset()")
print("=" * 60)
print(f"\nDeleting dataset: {DATASET_SLUG}\n")

delete_dataset_result = client.delete_dataset(DATASET_SLUG)
print(f"✓ Dataset {DATASET_SLUG} deleted successfully")
pprint(delete_dataset_result)


METHOD 20: delete_dataset()

Deleting dataset: reddit-traces-wrapper-demo-13

✓ Dataset reddit-traces-wrapper-demo-13 deleted successfully
{'success': True}


## Summary

### Congratulations! 🎉

You have successfully demonstrated all **20 methods** of the Noveum Platform API Client!

#### Methods Demonstrated:

**Trace Methods (6)**

1. `get_connection_status()` - Check API connection
2. `ingest_trace()` - Ingest single trace
3. `ingest_traces()` - Batch ingest traces
4. `query_traces()` - Query traces with filters
5. `get_trace()` - Get specific trace
6. `get_trace_spans()` - Get trace spans

**Dataset Methods (14)**

7. `create_dataset()` - Create new dataset
8. `list_datasets()` - List all datasets
9. `get_dataset()` - Get specific dataset
10. `update_dataset()` - Update dataset metadata
11. `create_dataset_version()` - Create dataset version
12. `list_dataset_versions()` - List dataset versions
13. `get_dataset_version()` - Get specific version
14. `get_dataset_versions_diff()` - Get diff between current and next release
15. `add_dataset_items()` - Add items to dataset
16. `publish_dataset_version()` - Publish version
17. `list_dataset_items()` - List dataset items
18. `delete_dataset_item()` - Delete specific item
19. `delete_all_dataset_items()` - Delete all items
20. `delete_dataset()` - Delete dataset



### Next Steps
- Customize the configuration variables at the top to use with your own data
- Explore different filter options in query methods
- Integrate these methods into your own workflows
- Check the [API documentation](../src/novaeval/noveum_platform/README.md) for more details
